## Data Cleaning

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from dotenv import load_dotenv
import os

load_dotenv()

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
def read_data(path):
    return pd.read_csv(path)

data_path = os.getenv("RAW_DATA")
df = read_data(data_path)

In [3]:
def check_df(dataframe):
    print("######### Head #########")
    print(df.head())
    print("######### Tail #########")
    print(df.tail())
    print("######### Info #########")
    print(df.info())
    print("######### Shape #########")
    print(df.shape)
    print("######### Null Values #########")
    print(df.isnull().sum())

check_df(df)

df.describe()

######### Head #########
    PLAKA  HATNO  HATKODU  DURAKSIRANO  DURAKID         DURAKGIRISTARIHI         DURAKCIKISTARIHI       HATBASLANGICTARIHI           HATBITISTARIHI               INSERTDATE
0  DB4218    800    28000           43    20060  2020-01-01 07:11:47.000  2020-01-01 07:12:17.000  2020-01-01 06:36:48.000  2020-01-01 07:23:24.000  2020-01-01 07:12:04.230
1  DB3476    800    28000           43    20060  2020-01-01 06:35:18.000  2020-01-01 06:35:44.000  2020-01-01 06:01:38.000  2020-01-01 06:45:11.000  2020-01-01 06:35:40.543
2  DB6224    800    28000           43    20060  2020-01-01 08:13:06.000  2020-01-01 08:13:38.000  2020-01-01 07:36:20.000  2020-01-01 08:23:16.000  2020-01-01 08:13:31.269
3  DB3476    800    28000           43    20060  2020-01-01 09:09:26.000  2020-01-01 09:09:54.000  2020-01-01 08:36:32.000  2020-01-01 09:18:52.000  2020-01-01 09:10:03.013
4  DB5969    800    28000           43    20060  2020-01-01 08:44:41.000  2020-01-01 08:44:53.000  2020-01-01 

,HATNO,HATKODU,DURAKSIRANO,DURAKID
count,561869.0,561869.0,561869.000000,561869.000000
mean,800.0,28000.0,42.906752,20096.983735
std,0.0,0.0,0.796924,37.000029
min,800.0,28000.0,42.000000,20060.000000
25%,800.0,28000.0,42.000000,20060.000000
50%,800.0,28000.0,43.000000,20060.000000
75%,800.0,28000.0,43.000000,20134.000000
max,800.0,28000.0,45.000000,20134.000000


In [4]:
# Drop rows with any missing values (e.g., from failed datetime parsing or duration calculation)
df = df.dropna()

In [5]:
def convert_to_datetime(dataframe, datetime_columns):
    """
    Converts specified columns in a DataFrame to datetime format.

    Parameters:
    ----------
    dataframe : pandas.DataFrame
        The DataFrame containing the columns to be converted.
    datetime_columns : list of str
        A list of column names in the DataFrame to convert to datetime format.

    Returns:
    -------
    pandas.DataFrame
        The original DataFrame with specified columns converted to datetime.
    """
    for col in datetime_columns:
        # Use 'coerce' to handle invalid parsing by setting them as NaT
        dataframe[col] = pd.to_datetime(dataframe[col], errors="coerce")
    return dataframe

# List of columns in `df` that are expected to contain datetime information
datetime_columns = [
    "DURAKGIRISTARIHI",
    "DURAKCIKISTARIHI",
    "HATBASLANGICTARIHI",
    "HATBITISTARIHI",
    "INSERTDATE",
]

# Apply the datetime conversion
df = convert_to_datetime(df, datetime_columns)

# Calculate route duration in minutes and store it in a new column
df["HATSURESI"] = (df["HATBITISTARIHI"] - df["HATBASLANGICTARIHI"]).dt.total_seconds() / 60


# Identify rows with non-positive durations
negative_duration = df[df["HATSURESI"] <= 0]

# Print the number of invalid duration rows
print(f"Number of non-positive durations: {negative_duration.shape[0]}")

# Optionally display those rows if any exist
if not negative_duration.empty:
    print(negative_duration)

# Remove rows with non-positive durations from the main DataFrame
df = df[df["HATSURESI"] > 0]


Number of non-positive durations: 7
             PLAKA  HATNO  HATKODU  DURAKSIRANO  DURAKID    DURAKGIRISTARIHI    DURAKCIKISTARIHI  HATBASLANGICTARIHI      HATBITISTARIHI              INSERTDATE     HATSURESI
38150    CI DB5385    800    28000           42    20060 2022-04-15 19:12:31 2022-04-15 19:14:39 2022-04-15 18:20:28 2022-04-15 17:22:11 2022-04-15 19:12:46.243    -58.283333
57601   CO1 BIA388    800    28000           42    20060 2022-12-07 09:41:42 2022-12-07 09:43:30 2022-12-07 08:45:48 2022-11-23 16:03:22 2022-12-07 09:42:11.303 -19722.433333
108711  CO2 BIA384    800    28000           42    20060 2023-04-22 09:20:47 2023-04-22 09:22:15 2023-04-22 09:31:21 2023-04-22 09:31:09 2023-04-22 09:20:58.646     -0.200000
272991   CI BIA622    800    28000           44    20060 2024-07-09 08:45:03 2024-07-09 08:45:45 2024-07-10 13:46:45 2024-07-09 09:03:09 2024-07-09 08:45:31.527  -1723.600000
319154   CI DB5385    800    28000           43    20134 2022-04-15 19:14:39 2022-04-15 1

#### Logical Outlier Setting

In [6]:
# Define logical lower and upper bounds for route duration (in minutes)
lower_bound = 35   #  routes shorter than 35 minutes are considered too short
upper_bound = 97   #  routes longer than 97 minutes are considered too long

# Identify rows with duration outside the logical bounds
logical_outliers = df[(df["HATSURESI"] < lower_bound) | (df["HATSURESI"] > upper_bound)]
print(f"Number of logical outliers: {logical_outliers.shape[0]}")

# Filter the DataFrame to keep only logically valid durations
df_cleaned = df[(df["HATSURESI"] >= lower_bound) & (df["HATSURESI"] <= upper_bound)]

# Remove rows where route start year is 2019
df_cleaned = df_cleaned[df_cleaned['HATBASLANGICTARIHI'].dt.year != 2019].reset_index(drop=True)

# Further filter: keep only rows where DURAKSIRANO is 43
df_cleaned = df_cleaned[df_cleaned["DURAKSIRANO"] == 43]


Number of logical outliers: 16425


In [7]:
# Apply Local Outlier Factor (LOF) to detect outliers based on 'HATSURESI'
lof = LocalOutlierFactor(n_neighbors=20)

# Fit the model and predict: -1 indicates outliers, 1 indicates inliers
y_pred = lof.fit_predict(df_cleaned[['HATSURESI']])

# Extract the rows identified as outliers
outliers_lof = df_cleaned[y_pred == -1]

# Keep only the rows identified as inliers (non-outliers)
df_cleaned= df_cleaned[y_pred != -1]

# Print the number of rows remaining after LOF cleaning
print(f"LOF Cleaned Dataset: {df_cleaned.shape[0]}")


LOF Cleaned Dataset: 193564


c:\Users\Excalibur\Desktop\Bitirme Projesi\Datas\800-last\IE-4912\venv\Lib\site-packages\sklearn\neighbors\_lof.py:322: UserWarning: Duplicate values are leading to incorrect results. Increase the number of neighbors for more accurate results.
  warnings.warn(


In [8]:
# Sort by route start time and reset index
df_cleaned = df_cleaned.sort_values(by='HATBASLANGICTARIHI').reset_index(drop=True)